In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Datasets/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
## remove nan values
df = df.dropna()

In [5]:
df.shape

(18285, 5)

In [6]:
x = df.drop('label',axis=1)

In [7]:
x.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [8]:

y = df['label']

In [9]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [10]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
voc_size = 5000

# One Hot Representation of Texts And Data Preprocessing such as Stopword Removal,Stemming

In [13]:
messages = x.copy()

In [14]:
messages.reset_index(inplace=True)

In [15]:
messages.head(10)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()
  
  #the words which are not in stopwards gonna get stemmed in porter stemmer
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] 
  review = ' '.join(review)
  corpus.append(review)


In [18]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [19]:
one_hot_repr = [one_hot(words,voc_size) for words in corpus]


# Word Embedding

In [20]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3194 1161 3828]
 [   0    0    0 ... 2598 1527 3200]
 [   0    0    0 ...   88 1199  397]
 ...
 [   0    0    0 ... 2616 4353 4540]
 [   0    0    0 ... 3695 2969 3923]
 [   0    0    0 ...  673  421 3253]]


In [21]:
len(embedded_docs)

18285

In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  698,
       3745, 2503, 4935, 4867, 2029, 1928, 3194, 1161, 3828], dtype=int32)

In [23]:
## Creating model
from tensorflow.keras.layers import Dropout
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            250000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 50)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(18285, (18285,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=0)

# Model Training

In [28]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 5s 10ms/step - loss: 0.3106 - accuracy: 0.8503 - val_loss: 0.1843 - val_accuracy: 0.9226
Epoch 2/10
229/229 [==============================] - 2s 8ms/step - loss: 0.1500 - accuracy: 0.9407 - val_loss: 0.1943 - val_accuracy: 0.9210
Epoch 3/10
229/229 [==============================] - 2s 9ms/step - loss: 0.1124 - accuracy: 0.9585 - val_loss: 0.2003 - val_accuracy: 0.9210
Epoch 4/10
229/229 [==============================] - 2s 9ms/step - loss: 0.0887 - accuracy: 0.9671 - val_loss: 0.2250 - val_accuracy: 0.9199
Epoch 5/10
229/229 [==============================] - 2s 9ms/step - loss: 0.0752 - accuracy: 0.9732 - val_loss: 0.2798 - val_accuracy: 0.9207
Epoch 6/10
229/229 [==============================] - 2s 9ms/step - loss: 0.0573 - accuracy: 0.9798 - val_loss: 0.2622 - val_accuracy: 0.9114
Epoch 7/10
229/229 [==============================] - 2s 9ms/step - loss: 0.0457 - accuracy: 0.9848 - val_loss: 0.3231 - val_accuracy: 0.9128
Epoch

# Performance Metrics and Accuracy

In [31]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(f"Accuracy \n {accuracy_score(y_pred,y_test)}")
print(f"Confusion matrix = \n {confusion_matrix(y_pred,y_test)}")
print(f"Accuracy\n {classification_report(y_pred,y_test)}")

Accuracy 
 0.9163248564397046
Confusion matrix = 
 [[1868  134]
 [ 172 1483]]
Accuracy
               precision    recall  f1-score   support

           0       0.92      0.93      0.92      2002
           1       0.92      0.90      0.91      1655

    accuracy                           0.92      3657
   macro avg       0.92      0.91      0.92      3657
weighted avg       0.92      0.92      0.92      3657



# Bidirectional LSTM

In [34]:
from tensorflow.keras.layers import Bidirectional
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               120800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 371,001
Trainable params: 371,001
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 6s 14ms/step - loss: 0.2959 - accuracy: 0.8658 - val_loss: 0.1994 - val_accuracy: 0.9163
Epoch 2/10
229/229 [==============================] - 3s 11ms/step - loss: 0.1429 - accuracy: 0.9428 - val_loss: 0.1932 - val_accuracy: 0.9226
Epoch 3/10
229/229 [==============================] - 3s 11ms/step - loss: 0.0994 - accuracy: 0.9631 - val_loss: 0.2188 - val_accuracy: 0.9199
Epoch 4/10
229/229 [==============================] - 3s 11ms/step - loss: 0.0688 - accuracy: 0.9754 - val_loss: 0.2541 - val_accuracy: 0.9171
Epoch 5/10
229/229 [==============================] - 3s 12ms/step - loss: 0.0527 - accuracy: 0.9825 - val_loss: 0.2986 - val_accuracy: 0.9218
Epoch 6/10
229/229 [==============================] - 3s 12ms/step - loss: 0.0389 - accuracy: 0.9873 - val_loss: 0.3121 - val_accuracy: 0.9182
Epoch 7/10
229/229 [==============================] - 3s 12ms/step - loss: 0.0261 - accuracy: 0.9924 - val_loss: 0.3774 - val_accuracy: 0.9150

In [36]:
y_pred1=model1.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [37]:
confusion_matrix(y_test,y_pred1)

array([[1867,  173],
       [ 139, 1478]])

In [38]:
accuracy_score(y_test,y_pred1)

0.9146841673502871

In [39]:
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      2040
           1       0.90      0.91      0.90      1617

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.92      0.91      0.91      3657

